## Primeiro escreva o caminho até a tabela de jogadores:

In [ ]:
file_path = '{ESCREVA O CAMINHO AQUI}'

## Apenas execute as células abaixo:

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso

In [ ]:
df = pd.read_csv(file_path)

non_numeric_cols = df[['full_name', 'position', 'Current Club']].copy()
df_numeric = df.select_dtypes(include=['number']).copy()

imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
df_numeric_imputed = pd.DataFrame(imp_mean.fit_transform(df_numeric), columns=df_numeric.columns)

data = pd.concat([non_numeric_cols, df_numeric_imputed], axis=1)

In [ ]:
# Função auxiliar para substituir a coluna de posição
def chance_of_goal(position):
    if position == 'Forward':
        return 1  # Alta chance de marcar gol
    elif position == 'Midfielder':
        return 0.7  # Chance intermediária
    elif position == 'Defender':
        return 0.3 # Baixa chance de marcar gol
    else:
        return 0 # Chance nula

# Normalizar as previsões para intervalo [0, 1] usando uma transformação sigmoide
def sigmoid(x):
    return np.round((1 / (1 + np.exp(-x))) * 100)

## Escolha os times que deseja prever:

In [ ]:
# Numerando os times para evitar possíveis erros de digitação nas testagens.
times = {
    0: 'Vitória',
    1: 'Flamengo',
    2: 'Cruzeiro',
    3: 'Botafogo',
    4: 'Grêmio',
    5: 'Fluminense',
    6: 'São Paulo',
    7: 'Palmeiras',
    8: 'Atlético Mineiro',
    9: 'Atlético PR',
    10: 'Corinthians',
    11: 'Vasco da Gama',
    12: 'Bahia',
    13: 'Atlético GO',
    14: 'Internacional', 
    15: 'Bragantino',
    16: 'Criciúma',
    17: 'Juventude',
    18: 'Cuiabá',
    19: 'Fortaleza'
}

In [ ]:
# Filtrar jogadores dos dois times específicos
time1 = times[1] 
time2 = times[11] 

### Agora execute o modelo:

In [ ]:
data_time1 = data[data['Current Club'] == time1]
data_time2 = data[data['Current Club'] == time2]
data_filtered = pd.concat([data_time1, data_time2])

X_filtered = data_filtered.select_dtypes(include=['number']).copy()
y_filtered = data_filtered['position'].apply(chance_of_goal)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_filtered)

model = Lasso(alpha=0.01, random_state=42)  
model.fit(X_scaled, y_filtered)
predictions = model.predict(X_scaled)

data_filtered['predicted_probability'] = sigmoid(predictions)
best = data_filtered.sort_values(by='predicted_probability', ascending=False)
best[['Current Club', 'full_name', 'predicted_probability']].head(10)